In [ ]:
import os
import workflow_utils_v3
import sys

from workflow_utils_v3.FileDirectory import Directory

dirs = Directory(rootpath = '/home/mgolub4/DLproj/MLTO_2024/')

# Sets directory of entire package
# rootpath = '/data/tigusa1/MLTO_UCAH/MLTO_2023/'

nbpath = os.path.join(dirs._3_Dynamic_PINN_RNN, 'PINN_training', 'arch1_singleLSTM')
cp_dir = os.path.join(nbpath, 'model_CPs')


In [ ]:

# from torchsummary import summary

import pandas as pd
import numpy as np
import json
import glob
import os

# For plotting
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from itertools import cycle
from plotly.colors import sequential, qualitative


import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F


import torch.utils.data as data
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import torch.optim as optim
# import torchsummary

# device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from sklearn.model_selection import train_test_split as TTS

In [ ]:
date = '30APR24'
fname_base = f'Dyn_PINN_v0_{date}'

In [ ]:
# dyndb_path = '/home/mgolub4/DLproj/MLTO_2024/3_Dynamic_PINN_RNN/dyn_data/dyn_stat_database_PINN_ready.csv'
dyndb_path = '/home/mgolub4/DLproj/MLTO_2024/3_Dynamic_PINN_RNN/dyn_data/dyn_stat_database_PINN_ready.csv'
dyndb = pd.read_csv(dyndb_path)

In [ ]:
# dyndb.columns.values

In [ ]:
# max_min_params = ['A_opt', 'B_opt', 'C_opt', 'm_opt', 'n_opt', 'plateau_stress_g', 'energy_absorbed_g']
# dict_entries = ['A_opt', 'B_opt', 'C_opt', 'm_opt', 'n_opt', 'sig_pl', 'W']


# Modifying from above after calculating sig_pl and W from scaled time series
max_min_params = ['A_opt', 'B_opt', 'C_opt', 'm_opt', 'n_opt'] # 'plateau_stress_g', 'energy_absorbed_g' <<-- in the revised version, these were calculated from the scaled stress/strain series, so pulling their column-wise max-min won't work
dict_entries = ['A_opt', 'B_opt', 'C_opt', 'm_opt', 'n_opt'] # 'sig_pl', 'W'
 
max_min_dict = {}

for par, name in zip(max_min_params, dict_entries):
    data = dyndb[par]
    max = data.max()
    min = data.min()
    max_min_dict[name] = (max, min) 

In [ ]:
idxTr, idxRem = TTS(dyndb, stratify = dyndb['topology_family'], random_state=42, train_size = 0.8)
idxVal, idxTe = TTS(idxRem, random_state = 42, test_size=0.5)

idxTr = idxTr.reset_index()
idxVal = idxVal.reset_index()
idxTe = idxTe.reset_index()


In [ ]:
params = ['volFrac', 
        'CH_11 scaled', 'CH_22 scaled', 'CH_33 scaled', 'CH_44 scaled', 'CH_55 scaled', 'CH_66 scaled',
        'CH_12 scaled', 'CH_13 scaled','CH_23 scaled',
        'EH_11 scaled', 'EH_22 scaled', 'EH_33 scaled',
        'GH_23 scaled', 'GH_13 scaled', 'GH_12 scaled', 
        'vH_12 scaled', 'vH_13 scaled', 'vH_23 scaled', 'vH_21 scaled', 'vH_31 scaled','vH_32 scaled',
        'KH_11 scaled', 'KH_22 scaled', 'KH_33 scaled', 
        'kappaH_11 scaled', 'kappaH_22 scaled', 'kappaH_33 scaled']

In [ ]:
class Stress_Series:
    def __init__(self, series):
        self.series = series
        self.max = series.max()
        self.min = series.min()

    def scale(self):
        return (self.series - self.min) / (self.max - self.min)

In [ ]:
import torch

def min_max_scale_series(batched_time_series):
    # Calculate the maximum and minimum values for each series
    max_values = torch.max(batched_time_series, dim=1)[0]
    min_values = torch.min(batched_time_series, dim=1)[0]

    # Calculate the range for each series
    series_range = max_values - min_values

    # Ensure non-zero range to avoid division by zero
    # series_range = torch.where(series_range == 0, torch.tensor(1e-7), series_range)

    # Min-max scale each series
    scaled_time_series = (batched_time_series - min_values.unsqueeze(1)) / series_range.unsqueeze(1)

    return scaled_time_series


In [ ]:
dyndb.loc[12, 'energy_absorbed_g scaled']

In [ ]:
class PINN_Dataset(Dataset):
    def __init__(self, params, split_dataframe,
                 feat_vec_directory='/home/mgolub4/DLproj/MLTO_2024/3_Dynamic_PINN_RNN/dyn_data/voxel_embedding_feature_maps', 
                 stress_series_directory='/home/mgolub4/DLproj/MLTO_2024/3_Dynamic_PINN_RNN/dyn_data/stress_series_data', 
                 stress_ser_suffix = '_proct_w_constit_eqn_and_scaled_series',
                 predicted_parameters=True,
                  ):
        
        self.df = split_dataframe
        self.featvec_dir = feat_vec_directory # for pulling the feature vectors
        self.stress_ser_dir = stress_series_directory # for pulling the time series files
        self.params = params
        self.predicted_parameters = predicted_parameters
        # self.const_eqn_params = ['A_opt', 'B_opt', 'C_opt', 'm_opt', 'n_opt',]
        self.const_eqn_params = ['A_opt scaled', 'B_opt scaled', 'C_opt scaled', 'm_opt scaled', 'n_opt scaled',]

        self.stress_ser_suffix = stress_ser_suffix
        # self.scale_coeffs_by = scale_coeffs_by

        # self.scaler = MinMaxScaler()


    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):

        dyn_series_fname = self.df['dyn_file_name_original'].iloc[idx]

        # sig_pl = self.df[self.df['dyn_file_name_original'] == dyn_series_fname]['plateau_stress_g scaled'].values[0].astype(np.float32)
        sig_pl = np.asarray(self.df.loc[idx, 'plateau_stress_g scaled'].astype(np.float32))

        # W = self.df[self.df['dyn_file_name_original'] == dyn_series_fname]['energy_absorbed_g scaled'].values[0].astype(np.float32)
        W = np.asarray(self.df.loc[idx, 'energy_absorbed_g scaled'].astype(np.float32))


        # feature vector from convolutional neural network convolutional layers output
        featvec_fname = self.df.loc[idx, 'conv_feat_vec']+ '.npy'
        featvec_path = os.path.join(self.featvec_dir, featvec_fname)
        featvec = np.load(featvec_path)
        featvec = np.squeeze(featvec, axis=0).astype(np.float32)

        # constitutive equation parameters
        # constit_eqn_coeffs = np.asarray(self.df[self.const_eqn_params].iloc[idx])

        # for i, scaler in enumerate(self.scale_coeffs_by):
        #     constit_eqn_coeffs[i] = constit_eqn_coeffs[i] / scaler
            


        # predicted parameters
        if self.predicted_parameters:
            paramvec = np.asarray([self.df.loc[idx, f'pred {par}'] for par in self.params]).astype(np.float32)
        else:
            paramvec = np.asarray([self.df.loc[idx, f'{par}']for par in self.params]).astype(np.float32)

        # stress_series -- for now (April 24), Imma use the truncated datasets, because I think padded batches for RNNs in pytorch will take care of differing lengths
        stress_ser_fname = dyn_series_fname + self.stress_ser_suffix
        stress_ser_path = os.path.join(self.stress_ser_dir, stress_ser_fname+'.csv')

        stress_series_data = np.asarray(pd.read_csv(stress_ser_path)['stress_bottom_gsreg_scaled']).astype(np.float32)
        stress_series_coneq = np.asarray(pd.read_csv(stress_ser_path)['stress_bottom_constitutive_equation_scaled']).astype(np.float32)
        
        # stress_series = Stress_Series(stress_series)
        # scaled_stress_series = stress_series.scale()
        # stress_series_dic = {'stress_series': stress_series.series, 'max': stress_series.max, 'min': stress_series.min}

        # stress_series_dic = {'stress_series': scaled_stress_series, 'max': stress_series.max, 'min': stress_series.min}

        strain = np.asarray(pd.read_csv(stress_ser_path)['Strain']).astype(np.float32)
        strain_end = len(strain)
        strain_end = np.float32(strain_end)
                   

        return featvec, paramvec, W, sig_pl, strain_end, strain, stress_series_data, stress_series_coneq#, idx
            #   0       1         2  3       4           5       6                   7



In [ ]:
# # Using this to check that the length of the original strain is the right ending index for the strain_end, to prevent (I hope, I pray...) negative W values from torch.trapz
# ls = [0,1,2,3,4,5]
# ls2 = [0,1,2,3,4,5,0]
# ln = len(ls)
# print(ls[:ln])
# print(ls2[:ln])

In [ ]:
# return featvec, paramvec, stress_series, constit_eqn_coeffs, W, sig_pl, strain
# return nonseries_list, padded_strain_ser, padded_stress_ser

from torch.nn.utils.rnn import pad_sequence

def padded_collate(batch):
    # get list of stress series -- uneven lengths, but will be padded at end
    stress_series_data = [torch.tensor(elem[-2], requires_grad=True) for elem in batch]
    stress_series_coneq = [torch.tensor(elem[-1], requires_grad=True) for elem in batch]

    # get max and min values as a list of lists, where the first value is the maximum and second is the minimum value of the accompanying stress series

    strn_ends = [item[4] for item in batch]
    # print(strn_ends)
    strn_ends = np.asarray(strn_ends)

    # get strain series, convert to tensor
    strain_ser = [torch.tensor(item[-3], requires_grad=True) for item in batch]
    
    # get other data (featvec, paramvec, constit_eqn_coeffs, W, sig_pl) from data
    nonserbatch = [item[0:-3] for item in batch] # all until last element (stress series) of list given by PINN_Dataset

    nonseries_list = []

    # nonser_len = len(nonserbatch)
    for i in range(5):
        data_list = []
        # for j, data in enumerate(nonserbatch):
        for data in nonserbatch:

            # print(j, type(data))
            # print(type(data))
            data_list.append(data[i])
        # print(data_list)
        data_list = torch.tensor(np.asarray(data_list), requires_grad=True)
        nonseries_list.append(data_list)

    
    stress_ser_data_padded = pad_sequence(stress_series_data, batch_first=True, padding_value=0)
    stress_ser_coneq_padded = pad_sequence(stress_series_coneq, batch_first=True, padding_value=0)

    padded_strain_ser = pad_sequence(strain_ser, batch_first=True, padding_value=0)


    return nonseries_list, padded_strain_ser, stress_ser_data_padded, stress_ser_coneq_padded, strn_ends

In [ ]:
batch_size = 8

trdata = PINN_Dataset(params, idxTr)
trloader = DataLoader(trdata, batch_size=batch_size, collate_fn = padded_collate, shuffle=True)

valdata = PINN_Dataset(params, idxVal)
valloader = DataLoader(valdata, batch_size=batch_size, collate_fn = padded_collate, shuffle=True)

tedata = PINN_Dataset(params, idxTe)
teloader = DataLoader(tedata, batch_size=batch_size, collate_fn = padded_collate, shuffle=False)

In [ ]:
class PINN_loss(nn.Module):

    def __init__(self, coeff_max_min_dict = max_min_dict, offset = -0.01):
        super(PINN_loss, self).__init__()

        self.offset = offset
        self.coeff_max_min_dict = coeff_max_min_dict



    def forward(self, stress_series_predicted, stress_series_data, stress_series_coneq, W_data, sig_pl_data, strain, strain_end): # , , stress_max_min_array, <<-- don't think I need these because I'm calculating the loss as scaled
        
        stress_data_loss = nn.L1Loss()(stress_series_predicted.squeeze(-1), stress_series_data)
        stress_coneq_loss = nn.L1Loss()(stress_series_predicted.squeeze(-1), stress_series_coneq)

        W_pred = self.predicted_W(stress_series_predicted, strain, strain_end)

        sig_pl_pred = self.predict_sig_pl(stress_series_predicted)

        # print(f"W_pred: {W_pred.detach().cpu().numpy()}\tSig_pl: pred - {sig_pl_pred.detach().cpu().numpy()}\t") #data - {sig_pl_data.detach().cpu().numpy()}")


        W_loss = nn.L1Loss()(W_pred, W_data)

        sig_pl_loss = nn.L1Loss()(sig_pl_pred, sig_pl_data)


        total_loss = stress_data_loss + stress_coneq_loss + W_loss + sig_pl_loss

        output_dic = {
            'total_loss': total_loss,
            'stress_data_loss': stress_data_loss,
            'stress_coneq_loss': stress_coneq_loss,
            'W_loss': W_loss,
            'sig_pl loss': sig_pl_loss,
            'predicted_sig_pl': sig_pl_pred, 
            'W_pred': W_pred, 
            'stress_series_predicted': stress_series_predicted, 
        }

        return total_loss, stress_data_loss, stress_coneq_loss, W_loss, sig_pl_loss, W_pred, stress_series_predicted, output_dic



    def unscale_coeffs(self, coeffs, coeff_max_min_dict, coeff_list =['A_opt', 'B_opt', 'C_opt', 'm_opt', 'n_opt',]):
        unscaled_coeffs = torch.zeros_like(coeffs)

        for i in range(coeffs.shape[0]):
            for j, name in enumerate(coeff_list):
                max = coeff_max_min_dict[name][0]
                min = coeff_max_min_dict[name][1]

                unscaled_coeffs[i][j] = (coeffs[i][j] * (max - min)) + min

        return unscaled_coeffs
    
    def unscale_dyn_params(array, coeff_max_min_dict, dyn_param = 'sig_pl'):
        max = coeff_max_min_dict[dyn_param][0]
        min = coeff_max_min_dict[dyn_param][1]

        array = array*(max - min) + min

        return array


    def predict_sig_pl(self, stress_series_predicted): # CALLED
         
        sig_pl_pred = torch.mean(stress_series_predicted[:, 200:400, 0], dim=1)

        return sig_pl_pred
    
    def predicted_W(self, stress_series_pred, strain, strain_end): # CALLED

        Ws_pred = []
        # print(stress_pred.shape)
        for i in range(stress_series_pred.shape[0]):
            strain_end_idx = int(strain_end[i])
            stress = stress_series_pred[i, :strain_end_idx, 0]
            strain_ser = strain[i, :strain_end_idx]
            # print(stress.shape, strain_ser.shape)
            W_pred = torch.trapz(stress, strain_ser)

            Ws_pred.append(W_pred)

        Ws_pred = torch.stack(Ws_pred, dim=0)

        return Ws_pred



In [ ]:
class Dynamic_Stress_PINN(nn.Module):
    
    def __init__(self, params, hidden_size=256, num_lstm_layers=4, stress_series_dim=1, coeff_dim=5):
        numparams = len(params)
        input_vec_dim = 1024 + numparams
        
        # self.series_in_dim = series_input_dim
        self.hidden_size = hidden_size
        self.num_lstm_layers = num_lstm_layers
        self.stress_series_dim = stress_series_dim
        self.coeff_dim = coeff_dim
        super(Dynamic_Stress_PINN, self).__init__()

        self.stress_ser_predictor = nn.LSTM(input_vec_dim, hidden_size, num_lstm_layers, batch_first=True)
        self.lstm_linear = nn.Sequential(nn.Linear(hidden_size, 128), nn.ReLU(), nn.Linear(128, stress_series_dim), nn.ReLU())

    def forward(self, feature_vec, property_vec, strain_series):
        input_vec = torch.cat([feature_vec, property_vec], dim=1)

        batch_size = strain_series.size(0)
        h = torch.randn(self.num_lstm_layers, self.hidden_size).to(strain_series.device)
        c = torch.randn(self.num_lstm_layers, self.hidden_size).to(strain_series.device)
        
        stress_ser = []

        for i in range(strain_series.size(1)):
            stress, (h, c) = self.stress_ser_predictor(input_vec, (h, c))

            stress = self.lstm_linear(stress)
            stress_ser.append(stress)
       
        stress_ser = torch.stack(stress_ser, dim=1)

        return stress_ser


In [ ]:
pinn = Dynamic_Stress_PINN(params).to(device)

In [ ]:
# from torchinfo import summary
# summary(pinn)


In [ ]:
# dataset returns-->         featvec, paramvec, constit_eqn_coeffs, W, sig_pl, || strain,            || stress_series,
# dataloader returns:->     ^--------------nonseries_list------------------^  || padded_strain_ser, || padded_stress_ser

# dataset    returns:   featvec, paramvec, constit_eqn_coeffs, W, sig_pl, strain,               stress_series_dic
# dataloader returns:   nonseries_list,                                   padded_strain_ser,    padded_stress_ser, max_mins


alpha = 1.0
beta = 1.0
gamma = 1.0
max_grad_norm = 1.0
def pinn_train(pinn, dataloader, loss_func, optimizer, alpha=alpha, beta=beta, gamma=gamma): #PINN_loss
    pinn.train()  # Set the model to training mode
    # running_loss = 0.0
    total_run_loss = 0.0
    stress_data_loss_run = 0.0
    stress_coneq_loss_run = 0.0
    # dyn_param_run_loss = 0.0
    W_loss_run = 0.0
    sig_pl_loss_run = 0.0
    pbar = tqdm(dataloader)  # Use tqdm for progress bars
    for non_series_data, strain_series, stress_series_data, stress_series_coneq, strain_ends in pbar:
        global feature_vec, param_vec, strn_ser_glob, strs_ser_data_glob, strs_ser_coneq_glob, strn_ends_glob
        feature_vec = non_series_data[0].cuda()
        param_vec   = non_series_data[1].cuda()
        strs_ser_data_glob = stress_series_data = stress_series_data.cuda()
        strs_ser_coneq_glob = stress_series_coneq = stress_series_coneq.cuda()
        strn_ends_glob = strain_ends

        W = non_series_data[2].cuda()
        # print(f"W shape: {W.shape}")
        sig_pl = non_series_data[3].cuda()
        strn_ser_glob = strain_series = strain_series.cuda()

    
        optimizer.zero_grad()
        global stress_series_pred

        stress_series_pred = pinn(feature_vec, param_vec, strain_series)
 
        global total_loss, stress_data_loss, stress_coneq_loss, W_loss, sig_pl_loss, W_pred, stress_series_predicted, output_dic
        total_loss, stress_data_loss, stress_coneq_loss, W_loss, sig_pl_loss, W_pred, stress_series_predicted, output_dic = loss_func()(stress_series_pred, stress_series_data, stress_series_coneq, W, sig_pl, strain_series, strain_ends)
                
        total_loss.backward() #retain_graph=True

        optimizer.step()
        # running_loss += loss_val.item()
        total_run_loss += total_loss.item()
        stress_data_loss_run += stress_data_loss.item()
        stress_coneq_loss_run += stress_coneq_loss.item()
        W_loss_run += W_loss.item()
        sig_pl_loss_run += sig_pl_loss.item()
        # pbar.set_description(f'Train Loss: {running_loss / (pbar.n + 1):.4f}')
        pbar.set_description(f'Losses:\tTotal: {total_run_loss / (pbar.n + 1):.4f}\tstress_data: {stress_data_loss_run / (pbar.n + 1):.4f}\tstress_coneq: {stress_coneq_loss_run / (pbar.n + 1):.4f}\tW: {W_loss_run / (pbar.n + 1):.4f}\tsig_pl: {sig_pl_loss_run / (pbar.n + 1):.4f}')
        # print("Data index is: f{idx}")
    return total_run_loss / len(dataloader), stress_data_loss_run / len(dataloader), stress_coneq_loss_run / len(dataloader),W_loss_run / len(dataloader), sig_pl_loss_run / len(dataloader)

In [ ]:
def pinn_validate(pinn, dataloader, loss_func=PINN_loss):
    pinn.train()  # Set the model to training mode
    # running_loss = 0.0
    total_run_loss = 0.0
    stress_data_loss_run = 0.0
    stress_coneq_loss_run = 0.0
    # dyn_param_run_loss = 0.0
    W_loss_run = 0.0
    sig_pl_loss_run = 0.0
    pbar = tqdm(dataloader)  # Use tqdm for progress bars
    with torch.no_grad():
        for non_series_data, strain_series, stress_series_data, stress_series_coneq, strain_ends in pbar:

            feature_vec = non_series_data[0].cuda()
            param_vec   = non_series_data[1].cuda()
            stress_series_data = stress_series_data.cuda()
            stress_series_coneq = stress_series_coneq.cuda()

            W = non_series_data[2].cuda()
            # print(f"W shape: {W.shape}")
            sig_pl = non_series_data[3].cuda()
            strain_series = strain_series.cuda()

            # global stress_series_pred

            stress_series_pred = pinn(feature_vec, param_vec, strain_series)
    
            total_loss, stress_data_loss, stress_coneq_loss, W_loss, sig_pl_loss, W_pred, stress_series_predicted, output_dic = loss_func()(stress_series_pred, stress_series_data, stress_series_coneq, W, sig_pl, strain_series, strain_ends)
        
            # running_loss += loss_val.item()
            total_run_loss += total_loss.item()
            stress_data_loss_run += stress_data_loss.item()
            stress_coneq_loss_run += stress_coneq_loss.item()
            W_loss_run += W_loss.item()
            sig_pl_loss_run += sig_pl_loss.item()
            # pbar.set_description(f'Train Loss: {running_loss / (pbar.n + 1):.4f}')
            pbar.set_description(f'Losses:\tTotal: {total_run_loss / (pbar.n + 1):.4f}\tstress_data: {stress_data_loss_run / (pbar.n + 1):.4f}\tstress_coneq: {stress_coneq_loss_run / (pbar.n + 1):.4f}\tW: {W_loss_run / (pbar.n + 1):.4f}\tsig_pl: {sig_pl_loss_run / (pbar.n + 1):.4f}')
            # print("Data index is: f{idx}")
    return total_run_loss / len(dataloader), stress_data_loss_run / len(dataloader), stress_coneq_loss_run / len(dataloader), W_loss_run / len(dataloader), sig_pl_loss_run / len(dataloader)


In [ ]:
# Model and training hyper(?)parameters
model = pinn
EPOCHS = 200

cp_dir = os.path.join(nbpath, 'model_CPs')

cp_name = f'CP_{fname_base}.pth'
best_weights_path = os.path.join(cp_dir, cp_name)
print(best_weights_path)

In [ ]:
patience = 75

min_val_loss = float('inf')
best_val_loss = float('inf')
early_stop_counter = 0
# earlystop_min_delta = 0.000075
earlystop_min_delta = 0.00075 # For L1Loss (MAE)

# os.makedirs(best_weights_path, exist_ok=True)
best_epoch = 0

train_losses = []
val_losses = []

epochs_completed=0

lrate = 1e-3
optimizer = optim.Adam(model.parameters(), lr=lrate, weight_decay=0.001)#optim.Adam(pinn.parameters(), lr=0.00001)
max_grad_norm = 1.0

In [ ]:

# return running_loss / len(dataloader),  dyn_param_loss, const_eqn_coeff_loss, phys_loss
lossfunc = PINN_loss
lossfunc_name = 'PINN_loss'

try:
#    return total_run_loss / len(dataloader), stress_data_loss_run / len(dataloader), stress_coneq_loss_run / len(dataloader), W_loss_run / len(dataloader), sig_pl_loss_run / len(dataloader)
    for epoch in range(EPOCHS):
        # Train the model
        train_loss = pinn_train(model, trloader, lossfunc, optimizer)
        tr_loss_val = train_loss[0]
        tr_data_loss = train_loss[1]
        tr_coneq_loss = train_loss[2]
        tr_W_loss = train_loss[3]
        tr_sigpl_loss = train_loss[4]
        

        # Validate the model
        val_loss = pinn_validate(model, valloader, lossfunc)
        val_loss_val = val_loss[0]
        val_data_loss = val_loss[1]
        val_coneq_loss = val_loss[2]
        val_W_loss = val_loss[3]
        val_sigpl_loss = val_loss[4]

        print(f'training:\ttotal loss: {tr_loss_val:.5f}, data loss: {tr_data_loss:.5f}\tconst. eqn series loss: {tr_coneq_loss:.5f}\tW loss: {tr_W_loss:.5f}\tplateau stress loss: {tr_sigpl_loss:.5f}')
        print(f'validation:\ttotal loss: {val_loss_val:.5f}, data loss: {val_data_loss:.5f}\tconst. eqn series loss: {val_coneq_loss:.5f}\tW loss: {val_W_loss:.5f}\tplateau stress loss: {val_sigpl_loss:.5f}')
        


        # Save the model's weights if validation loss is improved
        improvement_delta = best_val_loss - val_loss_val

        if val_loss_val < best_val_loss:
            pct_improved = (best_val_loss - val_loss_val) / best_val_loss * 100
            print(f"Val loss improved from {best_val_loss:.5f} to {val_loss_val:.5f} ({pct_improved:.2f}% improvement) saving model state...")
            best_val_loss = val_loss_val
            torch.save(model.state_dict(), best_weights_path)  # Save model weights to file
        else:
            print(f'Val loss did not improve from {best_val_loss:.5f}.')
            # early_stop_counter += 1  # Increment early stopping counter

        if improvement_delta > earlystop_min_delta:
            early_stop_counter = 0
        else:
            early_stop_counter +=1


        # Collect model training history
        train_losses.append(train_loss)
        val_losses.append(val_loss_val)

        # Check for early stopping
        if early_stop_counter >= patience:
            print(f'Validation loss did not improve for {early_stop_counter} epochs. Early stopping...')
            model.load_state_dict(torch.load(best_weights_path))
            print(f"Model best weights restored - training epoch {best_epoch}")
            break

        print(f'Epoch [{epoch+1}/{EPOCHS}]\tTrain Loss: {tr_loss_val:.5f}\tValidation Loss: {val_loss_val:.5f}')

        epochs_completed +=1


    # Load the best weights at end of training epochs
    model.load_state_dict(torch.load(best_weights_path))  # Load best model weights
    print(f'Training epochs completed, best model weights restored - epoch {best_epoch}')
    min_val_loss = best_val_loss

except KeyboardInterrupt:
    hist_dict = {f'train_loss {lossfunc_name}': train_losses, f'val_loss {lossfunc_name}': val_losses}
    model.load_state_dict(torch.load(best_weights_path))


In [ ]:

hist_dict = {f'train_loss {lossfunc_name}': train_losses, f'val_loss {lossfunc_name}': val_losses}

histno=1
histpath = os.path.join(nbpath,'model_jsons',f'{cp_name[:-4]}_training_history{histno}_{epochs_completed}ep.json')

model.eval()

with open(histpath, 'w') as f:
    json.dump(hist_dict, f)